In [15]:
import cv2
import numpy as np
import scipy.stats as st
import math

#gaussian kernel creation 
def gkern(kernlen=21, nsig=3):
    x = np.linspace(-nsig, nsig, kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kern2d = np.outer(kern1d, kern1d)
    return kern2d/kern2d.sum()


# Correlation fun (Image, Kernel , Kernel size, Padding, Number of channels)
def corrfun(img,kernel,ksize=3,p=1,nc=1):
    destimg = img
    if nc >1:
        for ch in range(nc):
            for i in range(p,img.shape[0]-p):
                for j in range(p,img.shape[1]-p):
                    fval = 0
                    k = math.floor(ksize/2)
                    for lt in range (i-k,i+k+1):
                        for wt in range (j-k,j+k+1):
                            #print(fval,img[lt][wt],kernel[lt-i+k][wt-j+k])
                            fval = fval + (img[lt][wt][ch])*(kernel[lt-i+k][wt-j+k])
                    #print(fval)
                    destimg[i][j][ch] = math.floor(fval)
    else:
        for ch in range(nc):
            for i in range(p,img.shape[0]-p):
                for j in range(p,img.shape[1]-p):
                    fval = 0
                    k = math.floor(ksize/2)
                    for lt in range (i-k,i+k+1):
                        for wt in range (j-k,j+k+1):
                            #print(fval,img[lt][wt],kernel[lt-i+k][wt-j+k])
                            fval = fval + (img[lt][wt])*(kernel[lt-i+k][wt-j+k])
                    #print(fval)
                    destimg[i][j] = math.floor(fval)
            
    return destimg

# Convolution fun(inverted kernel) (Image, Kernel , Kernel size, Padding, Number of channels)
def convfun(img,kernel,ksize=3,p=1,nc=1):
    destimg = img
    if nc >1: #channel based processing
        for ch in range(nc):
            for i in range(p,img.shape[0]-p):
                for j in range(p,img.shape[1]-p):
                    fval = 0
                    k = math.floor(ksize/2)
                    for lt in range (i-k,i+k+1):
                        for wt in range (j-k,j+k+1):
                            #print(fval,img[lt][wt],kernel[lt-i+k][wt-j+k])
                            fval = fval + (img[lt][wt][ch])*(kernel[ksize-(lt-i+k)-1][ksize-(wt-j+k)-1])
                    #print(fval)
                    destimg[i][j][ch] = math.floor(fval)
    else:
        for ch in range(nc):
            for i in range(p,img.shape[0]-p):
                for j in range(p,img.shape[1]-p):
                    fval = 0
                    k = math.floor(ksize/2)
                    for lt in range (i-k,i+k+1):
                        for wt in range (j-k,j+k+1):
                            #print(fval,img[lt][wt],kernel[lt-i+k][wt-j+k])
                            fval = fval + (img[lt][wt])*(kernel[ksize-(lt-i+k)-1][ksize-(wt-j+k)-1])
                    #print(fval)
                    destimg[i][j] = math.floor(fval)
            
    return destimg


# Median Filter (Image Filter size) 
def median_filter(data, filter_size):
    temp = []
    indexer = filter_size // 2
    for i in range(len(data)):
        for j in range(len(data[0])):
            for z in range(filter_size):
                if i + z - indexer < 0 or i + z - indexer > len(data) - 1:
                    for c in range(filter_size):
                        temp.append(0)
                else:
                    if j + z - indexer < 0 or j + indexer > len(data[0]) - 1:
                        temp.append(0)
                    else:
                        for k in range(filter_size):
                            temp.append(data[i + z - indexer][j + k - indexer])

            temp.sort()
            data[i][j] = temp[len(temp) // 2]
            temp = []
    return data

## Part 1: Convolutions - Mean, Gaussian, Sharpening on Lena.png

In [16]:
img = cv2.imread("lena.png")

In [17]:
cv2.imshow("Input image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Kernel size 3

In [112]:
#preparing kernels
avgkernel = np.ones((3, 3), dtype=np.float32)
avgkernel /= 9
gkernel = gkern(3,10)

In [113]:
pimg = np.zeros((512,512,3))
pimg= cv2.copyMakeBorder(img,1,1,1,1,cv2.BORDER_CONSTANT,value=0) #Padding
pimg.shape

(514, 514, 3)

#### Mean filter with kernel size 3 on Lena

In [114]:


avgimg = corrfun(pimg, avgkernel, 3,1,3) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Gaussian filter with kernel size 3 on Lena

In [115]:
pimg= cv2.copyMakeBorder(img,1,1,1,1,cv2.BORDER_CONSTANT,value=0) #Padding

gimg = corrfun(pimg, gkernel, 3,1,3) #Corrfun Gaussian
cv2.imshow("Gaussian img",gimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Sharpening filter with kernel size 3 on Lena

In [116]:
#our original padded image
pimg= cv2.copyMakeBorder(img,1,1,1,1,cv2.BORDER_CONSTANT,value=0)
#Getting details
details= pimg - gimg 

#Adding details to the img
sharpimg = pimg + 2*details
cv2.imshow("Sharp img",sharpimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Kernel size 5

In [79]:
#preparing kernels
avgkernel = np.ones((5, 5), dtype=np.float32)
avgkernel /= 25
gkernel = gkern(5,10)

In [80]:
pimg = np.zeros((512,512,3))
pimg= cv2.copyMakeBorder(img,2,2,2,2,cv2.BORDER_CONSTANT,value=0) #Padding
pimg.shape

(516, 516, 3)

#### Mean filter with kernel size 5 on Lena

In [83]:

avgimg = corrfun(pimg, avgkernel, 5,2,3) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Gaussian filter with kernel size 5 on Lena

In [84]:
pimg= cv2.copyMakeBorder(img,2,2,2,2,cv2.BORDER_CONSTANT,value=0) #Padding

gimg = corrfun(pimg, gkernel, 5,2,3) #Corrfun Gaussian
cv2.imshow("Gaussian img",gimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Sharpening filter with kernel size 5 on Lena

In [100]:
#our original padded image
pimg= cv2.copyMakeBorder(img,2,2,2,2,cv2.BORDER_CONSTANT,value=0)
#Getting details
details= pimg - gimg 

#Adding details to the img
sharpimg = pimg + 2*details
cv2.imshow("Sharp img",sharpimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Kernel size 9

In [22]:
#preparing kernels
avgkernel = np.ones((9, 9), dtype=np.float32)
avgkernel /= 81
gkernel = gkern(9,10)

In [23]:
pimg = np.zeros((512,512,3))
pimg= cv2.copyMakeBorder(img,4,4,4,4,cv2.BORDER_CONSTANT,value=0) #Padding
pimg.shape

(520, 520, 3)

#### Mean filter with kernel size 9 on Lena

In [24]:
avgimg = corrfun(pimg, avgkernel, 9,4,3) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Gaussian filter with kernel size 9 on Lena

In [25]:
pimg= cv2.copyMakeBorder(img,4,4,4,4,cv2.BORDER_CONSTANT,value=0) #Padding

gimg = corrfun(pimg, gkernel, 9,4,3) #Corrfun Gaussian
cv2.imshow("Gaussian img",gimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Sharpening filter with kernel size 9 on Lena

In [26]:
#our original padded image
pimg= cv2.copyMakeBorder(img,4,4,4,4,cv2.BORDER_CONSTANT,value=0)
#Getting details
details= pimg - gimg 

#Adding details to the img
sharpimg = pimg + 2*details
cv2.imshow("Sharp img",sharpimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
cv2.imshow("Details img",details)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Sharpening filter with kernel size 5 on Lena

In [106]:
cv2.imshow("details img",details)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Part 2: Correlation,Convolutions - Mean & Median Filter on Art.png

In [5]:
img = cv2.imread('art.png',0)

In [6]:
cv2.imshow("Input image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Kernel size 3

In [126]:
#preparing kernels
avgkernel = np.ones((3, 3), dtype=np.float32)
avgkernel /= 9
gkernel = gkern(3,10)

#### Correlation - Mean filter with kernel size 3 on Art

In [127]:
pimg= cv2.copyMakeBorder(img,1,1,1,1,cv2.BORDER_CONSTANT,value=0) #Padding

avgimg = corrfun(pimg, avgkernel, 3,1,1) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Convolution - Mean filter with kernel size 3 on Art

In [128]:
pimg= cv2.copyMakeBorder(img,1,1,1,1,cv2.BORDER_CONSTANT,value=0) #Padding

avgimg = convfun(pimg, avgkernel, 3,1,1) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Median Filter Kernel Size 3 on Art

In [131]:
pimg= cv2.copyMakeBorder(img,1,1,1,1,cv2.BORDER_CONSTANT,value=0) #Padding
medimg = median_filter(pimg,5)
cv2.imshow("Median img",medimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Kernel size 5

In [132]:
#preparing kernels
avgkernel = np.ones((5, 5), dtype=np.float32)
avgkernel /= 25
gkernel = gkern(5,10)

#### Correlation - Mean filter with kernel size 5 on Art

In [133]:
pimg= cv2.copyMakeBorder(img,2,2,2,2,cv2.BORDER_CONSTANT,value=0) #Padding

avgimg = corrfun(pimg, avgkernel, 5,2,1) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Convolution - Mean filter with kernel size 5 on Art

In [134]:
pimg= cv2.copyMakeBorder(img,2,2,2,2,cv2.BORDER_CONSTANT,value=0) #Padding

avgimg = convfun(pimg, avgkernel, 5,2,1) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Median Filter- Kernel size 5 on Art

In [135]:
pimg= cv2.copyMakeBorder(img,2,2,2,2,cv2.BORDER_CONSTANT,value=0) #Padding
median_filter(pimg,5)
cv2.imshow("Median img",medimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Kernel size 9

In [2]:
#preparing kernels
avgkernel = np.ones((9, 9), dtype=np.float32)
avgkernel /= 81
gkernel = gkern(9,10)

In [3]:
gkernel

array([[1.36196331e-29, 5.10633281e-23, 1.58338696e-18, 4.90210725e-16,
        2.70689038e-15, 4.90210725e-16, 1.58338696e-18, 5.10633281e-23,
        1.35209396e-29],
       [5.10633281e-23, 1.91448878e-16, 5.93650411e-12, 1.83791963e-09,
        1.01487926e-08, 1.83791963e-09, 5.93650411e-12, 1.91448878e-16,
        5.06933022e-23],
       [1.58338696e-18, 5.93650411e-12, 1.84080896e-07, 5.69907621e-05,
        3.14696799e-04, 5.69907621e-05, 1.84080896e-07, 5.93650411e-12,
        1.57191309e-18],
       [4.90210725e-16, 1.83791963e-09, 5.69907621e-05, 1.76441284e-02,
        9.74289606e-02, 1.76441284e-02, 5.69907621e-05, 1.83791964e-09,
        4.86658456e-16],
       [2.70689038e-15, 1.01487926e-08, 3.14696799e-04, 9.74289606e-02,
        5.37992139e-01, 9.74289606e-02, 3.14696799e-04, 1.01487926e-08,
        2.68727514e-15],
       [4.90210725e-16, 1.83791963e-09, 5.69907621e-05, 1.76441284e-02,
        9.74289606e-02, 1.76441284e-02, 5.69907621e-05, 1.83791964e-09,
        4.8

#### Correlation - Mean filter with kernel size 9 on Art

In [7]:
pimg= cv2.copyMakeBorder(img,4,4,4,4,cv2.BORDER_CONSTANT,value=0) #Padding

avgimg = corrfun(pimg, avgkernel, 9,4,1) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Convolution - Mean filter with kernel size 9 on Art

In [8]:
pimg= cv2.copyMakeBorder(img,4,4,4,4,cv2.BORDER_CONSTANT,value=0) #Padding

avgimg = convfun(pimg, avgkernel, 9,4,1) #Corrfun mean
cv2.imshow("avg img",avgimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Median Filter Kernel Size 9 on Art

In [11]:
pimg= cv2.copyMakeBorder(img,4,4,4,4,cv2.BORDER_CONSTANT,value=0) #Padding
medimg = median_filter(pimg,9)
cv2.imshow("Median img",medimg)
cv2.waitKey(0)
cv2.destroyAllWindows()